<!--HEADER-->
*Tutorial sobre Flux.jl - 2020/2 [- Ricardo M. S. Rosa (IM/UFRJ)](http://www.im.ufrj.br/rrosa)*

<!--BADGES-->
<a href="https://nbviewer.jupyter.org/github/rmsrosa/TutorialFlux/blob/main/notebooks/06.00-Algoritmos_treinamento.ipynb" target="_blank"><img align="left" src="https://img.shields.io/badge/view%20in-nbviewer-orange" alt="View in NBViewer" title="View in NBViewer"></a><a href="https://mybinder.org/v2/gh/rmsrosa/TutorialFlux/julia-env-for-binder?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Frmsrosa%252FTutorialFlux%26branch%3Dmain%26urlpath%3Dtree%252FTutorialFlux%252Fnotebooks/06.00-Algoritmos_treinamento.ipynb" target="_blank"><img align="left" src="https://mybinder.org/badge.svg" alt="Open in binder" title="Open in binder"></a><a href="https://nbviewer.jupyter.org/github/rmsrosa/TutorialFlux/blob/main/notebooks/slides/06.00-Algoritmos_treinamento.slides.html" target="_blank"><img align="left" src="https://img.shields.io/badge/view-slides-darkgreen" alt="View Slides" title="View Slides"></a>&nbsp;

<!--NAVIGATOR-->
[<- 5. Treinando redes neurais](05.00-Treinando_redes_neurais.ipynb) | [Página inicial](00.00-Pagina_inicial.ipynb) 

---


# Algoritmos de treinamento

* O que o `Flux.train!` faz é apenas executar um passo de um dado algoritmo de otimização.

* Vamos ver detalhes do `Flux.train!`.

* Assim como as opções de algoritmos.

In [4]:
using Flux
using Random
using Plots

## `Flux.train!`

* O `Flux.train!` é parte do módulo [Flux.Optimise](https://github.com/FluxML/Flux.jl/blob/master/src/optimise/Optimise.jl).

* Vejamos o [código do `Flux.train!`](https://github.com/FluxML/Flux.jl/blob/master/src/optimise/train.jl#L96):

```julia
function train!(loss, ps, data, opt; cb = () -> ())
  ps = Params(ps)
  cb = runall(cb)
  @progress for d in data
    try
      gs = gradient(ps) do
        loss(batchmemaybe(d)...)
      end
      update!(opt, ps, gs)
      cb()
    catch ex
      if ex isa StopException
        break
      elseif ex isa SkipException
        continue
      else
        rethrow(ex)
      end
    end
  end
end
```

### Detalhes

* [Params](https://github.com/FluxML/Zygote.jl/blob/master/src/compiler/interface.jl#L67) e [gradient](https://github.com/FluxML/Zygote.jl/blob/master/src/compiler/interface.jl#L57) são importados do [FluxML/Zygote.jl](https://github.com/FluxML/Zygote.jl), que é um pacote de **diferenciação automática**.

* `Params` é um tipo composto que guarda quais parâmetros e em que ordem devemos diferenciar uma função

* `params`, utilizado no treinamento visto anteriormente, define quais parâmetros serão considerados para a diferenciação.

* `gradient` calcula o gradiente de uma função, em relação aos parâmetros dados, via diferenciação automática.

* `gradient` usa `backwards propagation` (veja o uso de `pullback(f, args...)` na sua definição).

* **Observação:** Podemos "pular" o `Flux.train!` e escrever métodos de otimização com outros métodos de diferenciação, como `ForwardDiff` (forwards propagation) e `ReverseDiff` (backward mas mais genérico e não tão eficiente quando o `gradient` no caso de redes neurais em questão). Leia mais sobre isso e sobre outros pacotes em [JuliaDiff](https://juliadiff.org).

### Atalho para uma sequência de treinos

* É comum vermos o uso de 

```julia
@epochs N train!(...)
```

* Isso é apenas um atalho para um loop com a exibição de época a cada iteração:

```julia
for i=1:N
    train!(...)
end
```

### Callbacks

* O parâmetro de `callback` pode ser muito útil para
    * exibir informações sobre um *loop* de treinamento
    * afetar o treinamento de alguma forma
        * interromper quando um determinada acurácia é alcançada;
        * interromper quando um limite de iterações o de tempo é alcançado;
        * trocar os parâmetros correntes de busca caso o treinamente não esteja indo com muito sucesso;
        * substituir, acrescentar ou retirar determinados parâmetros do processo de treinamento;
        * validar o treinamento de acordo com outra amostra de dados;
        * etc.

* Vale lembrar que `callbacks` estão presentes em outros pacotes também, como `DifferentialEquations`, etc., para executar alguma instrução no meio de algum outro processo (e.g. resolução de uma EDO).

* O *Callback* é uma *keyword*, ou seja, é preciso passar como `cb = funcao_de_callback`, exceto que, no Julia 1.6, se a própria função tiver o nome da *keyword*, i.e. `cb`, no caso, então podemos passá-la direto como `Flux.train!(loss, ps, data, opt; cb)`.

* Veja mais em [Callback Helpers](https://fluxml.ai/Flux.jl/stable/utilities/#Callback-Helpers-1).

## Métodos de otimização

* Uma das opções, em `Flux.train!(loss, ps, data, opt, cb)` é o método de otimização `opt`.

* Não há opção *default* para o método de otimização. É preciso escolher um.

* Há várias opções.

* Todos os métodos estão implementados como diferentes "despachos" da função `Flux.Optimise.apply!`, em [src/optimise/optimisers.jl](https://github.com/FluxML/Flux.jl/blob/master/src/optimise/optimisers.jl).

* Essa função `apply!` é chamada no processo de atualização do passo, dentro da função [update!](https://github.com/FluxML/Flux.jl/blob/master/src/optimise/train.jl#L26), após o gradiente ter sido calculado. 

In [3]:
methods(Flux.Optimise.apply!)

# 19 methods for generic function "apply!":
[1] apply!(o::ClipNorm, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:666
[2] apply!(o::ClipValue, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:655
[3] apply!(o::AMSGrad, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:415
[4] apply!(o::RMSProp, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:139
[5] apply!(o::ADAGrad, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:346
[6] apply!(o::RADAM, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:213
[7] apply!(o::ADADelta, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:377
[8] apply!(o::WeightDecay, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:641
[9] apply!(o::ExpDecay, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:617
[10] apply!(o::ADAM, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:172
[11] apply!(o::AdaBelief, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:521
[12] apply!(o::Flux.Optimise.Optimiser, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:551
[13] apply!(o::Nesterov, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:102
[14] apply!(o::AdaMax, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:262
[15] apply!(o::InvDecay, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:577
[16] apply!(o::Descent, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:39
[17] apply!(o::NADAM, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:455
[18] apply!(o::Momentum, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:69
[19] apply!(o::OADAM, x, Δ) in Flux.Optimise at /Users/rrosa/.julia/packages/Flux/qp1gc/src/optimise/optimisers.jl:304

## Treinando em lotes o gradiente estocástico

* No caderno anterior, fizemos exemplos com passos "tradicionais", ou seja, levando-se em consideração todos os dados da amostra em cada "época".

* O método de gradiente estocástico utiliza apenas um dado da amostra em cada passo, escolhido aleatoriamente, completando uma "época" quando todos os dados são utilizados (no caso sem reposição) ou com algumas reutilizações e outros esquecios (no caso com reposição).

* E um caso intermediário é o **em lotes** *(batch)*, em que a amostra é dividida em grupos iguais escolhidos aleatoriamente e todos de um mesmo grupo são considerados a cada passo.

* Essa metodologia pode ser aplicada, na verdade, em conjunto com qualquer método de otimização, não apenas o de gradiente descendente.

* Para implementar isso, podemos fazer essa seleção manualmente ou usar a função 

## Inicialização das camadas

* A inicialização das camadas é um ponto importante do processo.

* Isso é particularmente importante em camadas convolucioais ou recorrentes.

* Por *default*, `Flux.jl` inicializa essas camadas com o método `Flux.glorot_uniform`.

* Outros métodos estão disponíveis: `glorot_normal`, `kalman_uniform`, `kalman_normal`.

* Mais sobre isso em [Layer Initialization](https://fluxml.ai/Flux.jl/stable/utilities/#Layer-Initialization-1).

<!--NAVIGATOR-->

---
[<- 5. Treinando redes neurais](05.00-Treinando_redes_neurais.ipynb) | [Página inicial](00.00-Pagina_inicial.ipynb) 